幅広くデータを見ていく
* 加速度の波形を並べていみる
* ウィンドウフレームごとの平均，分散の変化

In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [2]:
import numpy as np
from numpy import zeros, newaxis
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"

#chopin
#StorePath = "/home/takeyama/Documents/"
#mozart
StorePath ="/media/takeyama/HD-PNFU3/01_ActivityResearchData/"
if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/')
GraphPath=StorePath+'/'+TITLE+DATE+'graph/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/corrMap/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/corrMap/')
CorrMapGraphPath=StorePath+'/'+TITLE+DATE+'graph/corrMap/'

if not os.path.exists(StorePath+'/'+TITLE+'feature/'): 
    os.makedirs(StorePath+'/'+TITLE+'feature/')
FeaturePath=StorePath+'/'+TITLE+'feature/'

センサデータの読み込み

In [3]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する

In [4]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Generate WindowFrame Function
from mymodule import window

# timer
import time

# temp file 
import tempfile

%matplotlib inline

In [5]:
DictName = os.listdir(DictionaryDataPath)
print DictName

['Chest.npz', 'LeftHand.npz', 'LeftLeg.npz', 'RightHand.npz', 'Rightleg.npz', 'West.npz']


In [6]:
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

In [7]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

# Methematical/Statistical Functions

## Mean, Median

In [8]:
def FastVector_Mean(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].mean() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Mean==========='
    return Output

In [9]:
def FastVector_Median(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [np.median(wind[i,:]) for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Median==========='
    return Output

## Variance, Std Deviation

In [10]:
def FastVector_Var(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].var() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Variance==========='
    return Output

In [11]:
def FastVector_StdDev(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)  
    Output = np.array( [wind[i,:].var()**0.5 for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of standard Deviation==========='
    return Output

## Min, Max, Range
Range = the difference between maximum and minimum sample values

In [12]:
def FastVector_Max(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].max() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Max==========='
    return Output

In [13]:
def FastVector_Min(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].min() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Min==========='
    return Output

In [14]:
def FastVector_Range(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [(wind[i,:].max()-wind[i,:].min())\
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Range==========='
    return Output

## RMS

In [15]:
def FastVector_RMS(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [((np.sum( wind[i,:]**2)/wind[i,:].size)**0.5)\
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Root Means Square==========='
    return Output

## Correlation, Cross-Correlation

In [16]:
def FastVector_Corr(windowNameX, dataRawX, windowNameY, dataRawY, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowNameX, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowNameY, dataRawY, wWidth, sWidth, PATH)
    Output = np.array( [( np.corrcoef(windX[i,:],windY[i,:]) )\
                    for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Correlation==========='
    return Output[:,0,1]

# Signal magnitude area

In [17]:
def FastVector_SMA(windowName, dataRawX, dataRawY, dataRawZ, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowName, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowName, dataRawY, wWidth, sWidth, PATH)
    windZ=GetWindowFrame(windowName, dataRawZ, wWidth, sWidth, PATH)
    Output = np.array( [(np.sum((windX[i,:]**2+windY[i,:]**2+windZ[i,:]**2)**0.5))\
                for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Signal Magnitude Area==========='
    return Output  

# 各センサの相関を求める

In [18]:
DataList = np.array( [x[n]\
            for x in [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor5]\
            for n in ['AccX','AccY','AccZ']])
print DataList.shape

(18, 1077870)


In [19]:
NameArray = np.array([
         (x['Name'], y['Name']) for x in [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6] \
        for y in [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6]
    ])

In [20]:
NameArray

array([['LeftHand', 'LeftHand'],
       ['LeftHand', 'RightHand'],
       ['LeftHand', 'LeftLeg'],
       ['LeftHand', 'Rightleg'],
       ['LeftHand', 'Chest'],
       ['LeftHand', 'West'],
       ['RightHand', 'LeftHand'],
       ['RightHand', 'RightHand'],
       ['RightHand', 'LeftLeg'],
       ['RightHand', 'Rightleg'],
       ['RightHand', 'Chest'],
       ['RightHand', 'West'],
       ['LeftLeg', 'LeftHand'],
       ['LeftLeg', 'RightHand'],
       ['LeftLeg', 'LeftLeg'],
       ['LeftLeg', 'Rightleg'],
       ['LeftLeg', 'Chest'],
       ['LeftLeg', 'West'],
       ['Rightleg', 'LeftHand'],
       ['Rightleg', 'RightHand'],
       ['Rightleg', 'LeftLeg'],
       ['Rightleg', 'Rightleg'],
       ['Rightleg', 'Chest'],
       ['Rightleg', 'West'],
       ['Chest', 'LeftHand'],
       ['Chest', 'RightHand'],
       ['Chest', 'LeftLeg'],
       ['Chest', 'Rightleg'],
       ['Chest', 'Chest'],
       ['Chest', 'West'],
       ['West', 'LeftHand'],
       ['West', 'RightHand'],
    

In [21]:
test = GetWindowFrame('AccX', Sensor1['AccX'], 256, 32, WindowDataPath)
test.shape

this data had finished making


(33676, 256)

In [22]:
windowNum = 256
slidingNum = 32
CorrelatinMapAccX = np.array([
         FastVector_Corr(x['Name'], x['AccX'],y['Name'], y['AccX'], \
                         windowNum, slidingNum, WindowDataPath)
        for x in [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor5]
        for y in [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor5]
    ])

np.savez(FeaturePath+'w='+str(windowNum).zfill(4)+'_s='+str(slidingNum).zfill(4)\
         +'_CorrelationMapAccX',data=CorrelatinMapAccX)

this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data had finished making
===========finished Making Vector of Correlation===========
this data had finished making
this data 

In [23]:
CorrelatinMapAccX = CorrelatinMapAccX[:,:,newaxis]

In [24]:
print CorrelatinMapAccX.shape
CorrelatinMapAccX = CorrelatinMapAccX.reshape(6,6,33676)

(36, 33676, 1)


In [25]:
fil1 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(0,33676,9)])
fil2 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(1,33676,9)])
fil3 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(2,33676,9)])
fil4 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(3,33676,9)])
fil5 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(4,33676,9)])
fil6 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(5,33676,9)])
fil7 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(6,33676,9)])
fil8 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(7,33676,9)])
fil9 = np.array([CorrelatinMapAccX[:,:,i] for i in xrange(8,33676,9)])

In [26]:
fil1.shape

(3742, 6, 6)

In [27]:
fil1[0,:,:].shape

(6, 6)

In [35]:
fig = plt.figure(figsize=(15, 15))
fig = plt.figure()
fig.subplots_adjust(bottom = 0)
fig.subplots_adjust(top = 1)
fig.subplots_adjust(right = 1)
fig.subplots_adjust(left = 0)

X = xrange(6)
Y = xrange(6)

for i in tqdm_notebook( xrange(0,fil1.shape[0],9) ):
    ax1 = fig.add_subplot(331)
    ax1.set_title(str(i))
    cax1=ax1.imshow(fil1[i,:,:], interpolation='nearest')

    ax2 = fig.add_subplot(332)
    ax2.set_title(str(i+1))
    cax2=ax2.imshow(fil2[i,:,:], interpolation='nearest')

    ax3 = fig.add_subplot(333)
    ax3.set_title(str(i+2))
    cax3=ax3.imshow(fil3[i,:,:], interpolation='nearest')

    ax4 = fig.add_subplot(334)
    ax4.set_title(str(i+3))
    cax4=ax4.imshow(fil4[i,:,:], interpolation='nearest')

    ax5 = fig.add_subplot(335)
    ax5.set_title(str(i+4))
    cax5=ax5.imshow(fil5[i,:,:], interpolation='nearest')

    ax6 = fig.add_subplot(336)
    ax6.set_title(str(i+5))
    cax6=ax6.imshow(fil6[i,:,:], interpolation='nearest')

    ax7 = fig.add_subplot(337)
    ax7.set_title(str(i+6))
    cax7=ax7.imshow(fil7[i,:,:], interpolation='nearest')

    ax8 = fig.add_subplot(338)
    ax8.set_title(str(i+7))
    cax8=ax8.imshow(fil8[i,:,:], interpolation='nearest')

    ax9 = fig.add_subplot(339)
    ax9.set_title(str(i+8))
    cax9=ax9.imshow(fil9[i,:,:], interpolation='nearest')
    
    fig.tight_layout()
    fig.savefig(CorrMapGraphPath+'_'+str(i).zfill(4)+'_'+str(i+8).zfill(4))
    plt.cla()
    plt.clf()
    plt.close(fig)
    time.sleep(1)